# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. This is the dataset to use for this assignment. Note: The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [ ]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))
    
    print(df.head())

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

In [ ]:
#%matplotlib notebook
#%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.dates as dates
import numpy as np
import matplotlib.ticker as ticker

def plot_temperature_records():

    data = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')

    # converting data from tenths of degrees C to degrees C
    data['Data_Value'] = data['Data_Value']/10 
    
    # creating seperate Year and Month_Day field for convenience of later operations
    data['Date'] = pd.to_datetime(data['Date'])
    data['Year'] = data['Date'].dt.year
    data['Month_Day'] = data['Date'].dt.strftime('%m-%d')
    
    # removing leap days
    data = data[data['Month_Day'] != '02-29']

    # keeping data of 2005 to 2014 by removing 2015
    data_05_14 = data[(data['Year'] >= 2005) & (data['Year'] < 2015)]

    # record high and record low temperatures by day of the year over the period 2005-2014
    data_max = data_05_14[data_05_14['Element'] == 'TMAX'].groupby(['Month_Day'])['Data_Value'].agg('max')
    data_min = data_05_14[data_05_14['Element'] == 'TMIN'].groupby(['Month_Day'])['Data_Value'].agg('min')

    # merging high and low with actual dataset
    data = pd.merge(data, data_max.reset_index(drop=False).rename(columns={'Data_Value':'Max_Value'}), how='left', on='Month_Day')
    data = pd.merge(data, data_min.reset_index(drop=False).rename(columns={'Data_Value':'Min_value'}), how='left', on='Month_Day')

    # record high and record low was broken in 2015
    record_high = data[(data['Year']==2015) & (data['Data_Value'] > data['Max_Value'])]
    record_low = data[(data['Year']==2015) & (data['Data_Value'] < data['Min_value'])]

    # assigning X axis spanning using only 2015 
    date_index = np.arange('2015-01-01','2016-01-01', dtype='datetime64[D]')
    
    # defining figure size
    plt.figure(figsize=(15, 8))
    
    # plotting line graph of the record high and record low temperatures
    plt.plot(date_index, data_max, color='orange', linewidth=0.5)
    plt.plot(date_index, data_min, color='lightskyblue', linewidth=0.5)

    # scatter plot for record high or record low was broken in 2015
    plt.scatter(record_high['Date'].values, record_high['Data_Value'].values, color='darkred', s=10)
    plt.scatter(record_low['Date'].values, record_low['Data_Value'].values, color='midnightblue', s=10)

    ax = plt.gca()
    
    # shaded area between the record high and record low temperatures for each day
    ax.fill_between(date_index, data_max, data_min, facecolor='lavender', alpha=0.3, label='_nolegend_')
    
    # setting [Xmin,Xmax, Ymin,Ymax] limits
    ax.axis(['2015/01/01', '2015/12/31', -50, 50])
    
    # increasing Y axis ticks for better understanding of data  
    ax.locator_params(axis='y', nbins=10)
    
    # putting X axis ticker labels in minor ticks
    date_formatter = dates.DateFormatter('%b')
    ax.xaxis.set_major_locator(dates.MonthLocator())
    ax.xaxis.set_minor_locator(dates.MonthLocator(bymonthday=15))
    ax.xaxis.set_major_formatter(ticker.NullFormatter())
    ax.xaxis.set_minor_formatter(date_formatter)
    
    # removing minor ticks, keeping only ticker labels and major ticks. now ticker labels are in center  
    plt.tick_params(axis='x', which='minor', bottom=False, top=False, labelbottom=True) 

    # setting axises labels, title & legends
    plt.xlabel('Days of the Years', fontsize=12)
    plt.ylabel('Temperature (Celsius)', fontsize=12)
    plt.title('Temperature patterns near Ann Arbour, Michigan (2005-2015)', fontsize=15, fontweight="bold")
    plt.legend(['Record High (2005-2014)','Record Low (2005-2014)','Record Broken High of Days (2015)', 
                'Record Broken Low of Days (2015)'], loc='lower center', frameon=False, fontsize=12)

    # adding fahrenheit scale in second Y axis for convenience of understanding
    ax2 = ax.twinx()
    ax2.set_ylabel('Temperature (Fahrenheit)', fontsize=12)
    fahrenheit = lambda celsius: celsius*1.8 + 32
    ymin, ymax = ax.get_ylim()
    ax2.set_ylim((fahrenheit(ymin), fahrenheit(ymax)))
    
    plt.savefig('assignment_2.png')
    
    return plt.show()


In [ ]:
plot_temperature_records()
